In [5]:
from webscraping import *
from leagues import *
from fake_useragent import UserAgent # pip install fake_useragent

user_agent = UserAgent()
header = {'User-Agent': user_agent.random}
#amanha = datetime.strftime(datetime.today(),'%d-%m-%Y')
amanha = datetime.strftime(datetime.today() + timedelta(days=1),'%Y-%m-%d')
partidas_amanha = TinyDB(f'./Jogos_do_Dia/{amanha}.json')
warnings.filterwarnings('ignore')

options = webdriver.ChromeOptions()
options.add_argument(f'user-agent={header["User-Agent"]}')
options.add_argument('--headless')
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
driver.maximize_window()

url = 'https://www.flashscore.com/'
driver.get(url)
try:
    WebDriverWait(driver, 8).until(EC.visibility_of_element_located((By.CSS_SELECTOR,'button#onetrust-accept-btn-handler')))
    button_cookies = driver.find_element(By.CSS_SELECTOR,'button#onetrust-accept-btn-handler')
    button_cookies.click()
except:
    print("cookies already closed")

WebDriverWait(driver, 8).until(EC.element_to_be_clickable((By.CSS_SELECTOR,'.calendar__navigation--tomorrow')))
driver.execute_script("document.getElementsByClassName('otPlaceholder')[0].style.display='none';")
driver.find_element(By.CSS_SELECTOR,'.calendar__navigation--tomorrow').click()
sleep(5)

jogos = driver.find_elements(By.CSS_SELECTOR,'div.event__match--twoLine')
id_jogos =[]
for i in jogos:
    id_jogos.append(i.get_attribute("id")[4:])

print(driver.title)
for id_jogo in tqdm(id_jogos):
    procura_duplicado = Query()
    if partidas_amanha.search(procura_duplicado.Id == id_jogo) == []:
        driver.get(f'https://www.flashscore.com/match/{id_jogo}/#/match-summary/match-summary')
        country = driver.find_element(By.CSS_SELECTOR,'span.tournamentHeader__country').text.split(':')[0]
        league = driver.find_element(By.CSS_SELECTOR,'span.tournamentHeader__country > a').text.split(' -')[0]
        if f'{country} - {league}' in leagues_list:
            jogo = {}
            
            get_main_header(id_jogo,jogo,driver)
            get_odds_ml_ft(id_jogo,jogo,driver)
            get_odds_ou_ft(id_jogo,jogo,driver)
            get_odds_btts_ft(id_jogo,jogo,driver)
            get_odds_cs_ft(id_jogo,jogo,driver)

            partidas_amanha.insert(jogo.copy())
            
driver.close()
df = pd.DataFrame(partidas_amanha.all())


EURO U21 2023, Football Live Scores, Latest Football Results | Flashscore.com


  0%|          | 0/149 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
df = df[['Date', 'Time', 'League', 'Home', 'Away', 'Round', 'Odd_H', 'Odd_D', 'Odd_A', 'Odd_Over25', 'Odd_Under25', 'Odd_BTTS_Yes', 'Odd_BTTS_No']]

df.columns= ['Date','Time', 'League', 'Home', 'Away', 'Round', 'FT_Odd_H','FT_Odd_D','FT_Odd_A','FT_Odd_Over25','FT_Odd_Under25','FT_Odd_BTTS_Yes','FT_Odd_BTTS_No']

df.to_csv(f'./Jogos_do_Dia/{amanha}_Jogos_do_Dia.csv',index=False)

print('encerrado')